### Опрацювання та підготовка даних

- читаємо файли з правильним кодуванням та роздільниками
- приводимо url до нижнього регістру
- форматування акордів у вигляді "main accords: ..."
- видаляємо старі mainaccord-колонки, якщо є
- зберігаємо новий файл fra_main.csv

In [ ]:
import pandas as pd
import ast
import os
from dotenv import load_dotenv
load_dotenv()
WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")
OPENROUTER_KEY = os.getenv("OPENROUTER_KEY")
TOKEN = os.getenv("TOKEN")

fra_perfumes = pd.read_csv("data/fra_perfumes.csv", encoding="utf-8")
fra_cleaned = pd.read_csv("data/fra_cleaned.csv", sep=";", encoding="latin1")

fra_perfumes["url"] = fra_perfumes["url"].str.lower()
fra_cleaned["url"] = fra_cleaned["url"].str.lower()

def format_accords(accord_str):
    try:
        accords = ast.literal_eval(accord_str)
        if isinstance(accords, list):
            return "main accords: " + ", ".join(accords)
    except:
        pass
    return ""

fra_perfumes["Main Accords"] = fra_perfumes["Main Accords"].apply(format_accords)
merged = fra_cleaned.merge(
    fra_perfumes[["url", "Main Accords"]],
    on="url",
    how="left"
)

columns_to_drop = ["mainaccord1", "mainaccord2", "mainaccord3", "mainaccord4", "mainaccord5"]
merged = merged.drop(columns=[c for c in columns_to_drop if c in merged.columns])


merged.to_csv("data/fra_main.csv", sep=";", index=False, encoding="utf-8")
print("збережено")

- у fra_main.csv всі колонки зʼєднати і отримуємо Description
- створюємо базове речення опису:
    1. Назва + призначення: "The {Perfume} is designed for {Gender}."
    2. Бренд, країна, рік: "Brand {Brand}, {Country}, created in {Year}."
    3. Топ ноти: "Top notes are {Top}."
    4. Середні ноти: "Middle notes are {Middle}."
    5. Базові ноти: "Base notes are {Base}."
    6. Головні акорди: "Main accords include {Main Accords}."
- зберігаємо файл як fra_working.csv

In [ ]:
fra_main = pd.read_csv("data/fra_main.csv", sep=";", encoding="utf-8")

def make_description_en(row):
    parts = []
    
    if pd.notna(row["Perfume"]):
        text = f"The {row['Perfume']}"
        if pd.notna(row["Gender"]):
            text += f" is designed for {row['Gender']}"
        parts.append(text + ".")
    
    brand_country_year = []
    if pd.notna(row["Brand"]):
        brand_country_year.append(f"Brand {row['Brand']}")
    if pd.notna(row["Country"]):
        brand_country_year.append(row["Country"])
    if pd.notna(row["Year"]):
        brand_country_year.append(f"created in {int(row['Year'])}")
    if brand_country_year:
        parts.append(", ".join(brand_country_year) + ".")
    
    if pd.notna(row["Top"]):
        parts.append(f"Top notes are {row['Top']}.")
    
    if pd.notna(row["Middle"]):
        parts.append(f"Middle notes are {row['Middle']}.")
    
    if pd.notna(row["Base"]):
        parts.append(f"Base notes are {row['Base']}.")
    
    if pd.notna(row["Main Accords"]):
        accords = row["Main Accords"].replace("main accords:", "").strip()
        parts.append(f"Main accords include {accords}.")
    
    return " ".join(parts)

fra_main["Description"] = fra_main.apply(make_description_en, axis=1)
fra_main.to_csv("data/fra_working.csv", sep=";", index=False, encoding="utf-8")

print("збережено")


кінцевий файл для використання далі, залишаємо тільки url та description

In [ ]:
df = pd.read_csv("data/fra_working.csv", sep=";")
df_clean = df[['url', 'Description']]
df_clean.to_csv("data/fra_final.csv", sep=";", index=False)

print("збережено")


### Перетворення у векторну датабазу
- 1 вектор на discription
- генерація векторів через ембедингову модель sentence-transformers/all-MiniLM-L6-v2 (далі буде покращено з BAAI)
- отримуємо [Вектори ароматів]
- збереження векторів у векторну базу даних weaviate

In [ ]:
pip install sentence-transformers weaviate-client

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model_faster = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
df = pd.read_csv("data/fra_final.csv", delimiter=";")
print(df.head())

url_embeddings = model_faster.encode(df["url"].tolist(), convert_to_numpy=True)
desc_embeddings = model_faster.encode(df["Description"].tolist(), convert_to_numpy=True)

df["url_embedding"] = url_embeddings.tolist()
df["desc_embedding"] = desc_embeddings.tolist()

In [ ]:
df.head()

### Збереження векторів у векторну базу даних weaviate
- Підключення до Weaviate Cloud
- Створення колекції (Perfume) (+ перевірка чи існує і очищення, щоб не було збою при повторному завантаженні)
- Додавання даних у колекцію: Для кожного рядка додаємо властивості (url, description) і готовий вектор (desc_embedding)

In [ ]:
pip install weaviate-client

In [ ]:
import os
import weaviate
from weaviate.classes.init import Auth
from dotenv import load_dotenv

load_dotenv()

weaviate_url = ""
weaviate_api_key = ""

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key)
)

print(client.is_ready())

In [ ]:
from weaviate.classes.config import Property, DataType, Configure

if client.collections.exists("Perfume"):
    client.collections.delete("Perfume")

client.collections.create(
    name="Perfume",
    description="Perfume DB with one desc vector",
    properties=[
        Property(name="url", data_type=DataType.TEXT),
        Property(name="description", data_type=DataType.TEXT),
    ],
    vector_config={
        "name": "desc_embedding",
        "vectorizer": Configure.Vectorizer.none(),
        "vectorIndexConfig": Configure.VectorIndex.hnsw()._to_dict()
    }
)

print("Колекція Perfume створена!")


In [ ]:
collection = client.collections.get("Perfume")

In [ ]:
with collection.batch.fixed_size(100) as batch:
    for _, row in df.iterrows():
        batch.add_object(
            properties={
                "url": row["url"],
                "description": row["Description"],
            },
            vector={  
                "desc_embedding": row["desc_embedding"]
            }
        )

print("Дані додані в колекцію")

In [ ]:
print("Кількість об'єктів:", collection.aggregate.over_all(total_count=True).total_count)

### Обробка запиту користувача та формування відповіді

- Визначення запиту користувача
- Перетворення тексту на вектор (embedding з тією ж моделлю)
- Пошук найближчих топ-5 векторів у колекції до "вектора користувача": Weaviate шукає топ-5 найбільш схожих векторів у колекції за алгоритмом approximate nearest neighbor (ANN). За замовчуванням метрика схожості cosine similarity. Список вже ранжований від найбільш до менш схожого, тобто перший елемент — найбільш релевантний.
- Фільтр відповідей, якщо мінімальна схожість (certainty) менша 0.5 по cosine similarity, то цей парфум у відповідь не виводиться
- Виведення знайдених результатів, URL та Description

In [ ]:
query = "vanilla floral perfume"
q_vec = model_faster.encode([query])[0].tolist()

top_n = 5
threshold = 0.5

result = collection.query.near_vector(
    near_vector=q_vec,
    target_vector="desc_embedding",
    limit=top_n,
    return_properties=["url", "description"],
    return_metadata=["distance"]
)

filtered_results = []
for obj in result.objects:
    distance = obj.metadata.distance
    if distance is not None:
        similarity = 1 - distance
        if similarity >= threshold:
            filtered_results.append((obj, similarity))

for obj, sim in filtered_results:
    print(f"URL: {obj.properties['url']}")
    print(f"Description: {obj.properties['description']}")
    print(f"Cosine similarity: {sim:.3f}")
    print("-" * 60)


Зберігання API ключа

In [ ]:
os.environ["OPENROUTER_KEY"] = (
    ""
)
OPENROUTER_KEY = os.environ["OPENROUTER_KEY"]

- Готуємо запит до API (модель claude-3.5-sonnet)
- Відправляємо запит і отримуємо відповідь
- Виводимо відповідь

In [ ]:
import requests

headers = {
    "Authorization": f"Bearer {OPENROUTER_KEY}",
    "Content-Type": "application/json",
}

data = {
    "model": "anthropic/claude-3.5-sonnet",
    "messages": [
        {"role": "user", "content": "Привіт, хто ти?"}
    ]
}

response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
print(response.json()["choices"][0]["message"]["content"])

### Пошуку та уточнення запиту
- Перевірка запиту на зміст
- Основна функція пошуку

In [ ]:
def check_user_query(query: str) -> str:
    headers = {
        "Authorization": f"Bearer {OPENROUTER_KEY}",
        "Content-Type": "application/json",
    }

    data = {
        "model": "anthropic/claude-3.5-sonnet",
        "max_tokens": 200,
        "messages": [
            {
                "role": "system",
                "content": (
                    "Ти асистент з підбору парфумів.\n\n"
                    "Твоє завдання — визначити, чи містить запит користувача "
                    "інформацію, пов’язану з ароматами.\n\n"
                    "Вважай ЗАПИТ КОРЕКТНИМ, якщо є ХОЧА Б ОДНЕ з наступного:\n"
                    "- конкретні ноти (ваніль, троянда, мускус, citrus, cherry тощо)\n"
                    "- aroma-групи (солодкий, свіжий, деревний, квітковий, фруктовий, морський тощо)\n"
                    "- семантичний опис запаху (затишний, як ліс, запах моря, теплий, чистий і т.д.)\n"
                    "- вказівка на стать аромату (жіночий, чоловічий, унісекс, women, men)\n\n"
                    "Враховуй:\n"
                    "- українську й англійську\n"
                    "- опечатки\n"
                    "- синоніми\n\n"
                    "Якщо запит КОРЕКТНИЙ — відповідай ТІЛЬКИ словом:\n"
                    "ok\n\n"
                    "Якщо інформації НЕДОСТАТНЬО — задай КОРОТКЕ уточнення українською "
                    "(до 15 слів), наприклад:\n"
                    '- "Можеш описати аромат або назвати ноти?"\n'
                    '- "Тобі більше солодкий, свіжий чи деревний аромат?"\n\n'
                    "Не пояснюй рішення\n"
                    "Не аналізуй текст\n"
                    "Не повторюй запит"
                ),
            },
            {"role": "user", "content": query},
        ],
    }

    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data,
            timeout=15,
        )
        response.raise_for_status()
        result = response.json()
        return result["choices"][0]["message"]["content"].strip()

    except Exception as e:
        print("Claude error:", e)
        return "Можеш трохи уточнити, який аромат шукаєш?"

In [ ]:
def normalize_to_english_notes(query: str) -> str:
    headers = {
        "Authorization": f"Bearer {OPENROUTER_KEY}",
        "Content-Type": "application/json",
    }

    data = {
        "model": "anthropic/claude-3.5-sonnet",
        "max_tokens": 50,
        "messages": [
            {
                "role": "system",
                "content": (
                    "Твоя задача — нормалізувати запит користувача про парфуми. "
                    "Переклади запит на англійську мову та заміни всі ароматичні ноти, "
                    "акорди й характеристики на стандартні англійські назви, "
                    "які використовуються у парфумерних базах даних. "
                    "Виправляй опечатки, синоніми та різні варіанти написання. "
                    "Обробляй увесь запит цілком, незалежно від мови оригіналу. "
                    "У відповіді поверни тільки нормалізований англійський текст "
                    "без пояснень, коментарів або будь-якого іншого тексту."
                ),
            },
            {"role": "user", "content": query},
        ],
    }

    response = requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers=headers,
        json=data,
        timeout=15,
    )
    response.raise_for_status()
    result = response.json()
    return result["choices"][0]["message"]["content"].strip()


def encode_user_query(full_query: str, model):
    normalized_query = normalize_to_english_notes(full_query)
    vector = model.encode([normalized_query])[0].tolist()
    return vector

In [ ]:
def search_in_weaviate(vector, collection, k=5):
    result = collection.query.near_vector(
        near_vector=vector,
        target_vector="desc_embedding",
        limit=k,
        return_properties=["url", "description"],
    )
    return result.objects

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


def rerank_results(vector, results, model):
    sims = []
    for obj in results:
        desc = obj.properties["description"]
        vec = model.encode([desc])[0].reshape(1, -1)
        sim = cosine_similarity([vector], vec)[0][0]
        sims.append((sim, obj))

    sims.sort(key=lambda x: x[0], reverse=True)
    return [o for _, o in sims]

In [ ]:
def generate_response(
    query: str, results, model_name="anthropic/claude-3.5-sonnet", top_k=5
):

    if not results:
        return "😕 Нічого не знайдено. Спробуй описати аромат інакше."

    context_docs = []
    for obj in results[:top_k]:
        url = obj.properties.get("url", "")
        desc = obj.properties.get("description", "")
        context_docs.append(f"{url}\n{desc}")

    context = "\n\n".join(context_docs)

    rag_prompt = f"""
Ти — корисний асистент, який допомагає підібрати аромат парфумів.

Перед аналізом контексту інтерпретуй запит користувача та внутрішньо
нормалізуй усі ароматичні ноти до стандартних англійських назв,
які використовуються у базі Fragrantica. Слова «вишня», «вишневий»,
«черрі» та «cherry» трактуй як cherry. Застосовуй цю логіку до всіх
ароматичних нот незалежно від мови написання або опечаток.
Використовуй для підбору лише цю нормалізовану англійську інтерпретацію
та не пояснюй процес нормалізації у відповіді.

Категорично заборонено рекомендувати або згадувати аромати, бренди,
будинки або описи, пов’язані з Росією, Москвою або російськими компаніями,
незалежно від мови написання. Такі аромати мають бути повністю виключені
з результатів, навіть якщо вони добре підходять за нотами.

Перед використанням контексту уяви, що пошук ароматів
виконувався не лише за оригінальним запитом користувача,
а й за його англійською ароматичною інтерпретацією.

Якщо користувач використовує слова іншою мовою,
вважай, що пошук також включає відповідні англійські ноти.
Наприклад, запит «вишневий» слід інтерпретувати так,
ніби пошук виконувався за словом cherry.

Відповідай коротко пояснючи, які варіанти підійшли і чому.
Для кожного аромату обовʼязково вкажи пряме посилання на відповідну сторінку
на сайті https://www.fragrantica.com, якщо URL присутній у контексті.
Не вигадуй аромати і не додавай неперевірену інформацію.
Не повторюй текст запиту користувача і не пояснюй процес підбору.

Контекст (URL + опис):
{context}

Запит користувача:
{query}
"""

    headers = {
        "Authorization": f"Bearer {OPENROUTER_KEY}",
        "Content-Type": "application/json",
    }

    data = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": "Ти корисний ароматний асистент."},
            {"role": "user", "content": rag_prompt},
        ],
        "temperature": 0.4,
        "max_tokens": 300,
    }

    try:
        r = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data,
            timeout=15,
        )
        r.raise_for_status()
        result = r.json()
        llm_answer = result["choices"][0]["message"]["content"].strip()
        return f"Відповідь:\n{llm_answer}"

    except (requests.exceptions.RequestException, KeyError) as e:
        print(f"Помилка під час запиту до LLM: {e}")
        return "Не вдалося обробити запит, спробуйте ще раз."

In [ ]:
def handle_user_message_kate(message: str, model, collection):

    clarifying = check_user_query(message)
    if clarifying.lower() != "ok":
        return clarifying

    user_vec = encode_user_query(message, model)
    raw_results = search_in_weaviate(user_vec, collection, k=5)
    ranked = rerank_results(user_vec, raw_results, model)
    answer = generate_response(message, ranked)
    return answer


# print(
#     handle_user_message_kate(
#         "Generate something with vanilla and strawberry for me",
#         model_faster,
#         collection,
#     )
# )

- Обробляє текст користувача (query), який треба перевірити на наявність опису ароматичних нот.
- Формує запит до моделі anthropic/claude-3.5-sonnet через OpenRouter
- Відправляємо запит на сервер і чекаємо відповідь (не більше 15 секунд)
- Отримує відповідь від моделі у JSON-форматі.
- Повертає тільки текст відповіді — 'ok', якщо опис є, або уточнення українською, якщо інформації бракує. Якщо щось пішло не так (помилка мережі або API), то повертає повідомлення про помилку.

### Завантаження даних та витяг потенційних нот із датасету
Цей блок читає файл fra_working.csv, витягує тексти з колонок Top/Middle/Base/Main Accords,
нормалізує їх, знаходить повторювані двослівні та однословні комбінації,
і будує глобальний список EXPLICIT_NOTES — набір усіх нот, які реально присутні у базі.
Саме цей список дозволяє боту визначити, чи користувач назвав реальні ноти.

### Словник ароматичних підкатегорій (потрібний для уточнюючих питань)
Цей блок створює структуроване дерево ароматичних груп,
яке використовується для генерації уточнювальних питань,
коли запит користувача нечіткий (типу: “хочу щось солодке” або “щось свіже”).

### Попередня обробка тексту користувача та визначення груп ароматів
Цей блок:
- нормалізує текст
- визначає, чи містить він реальні ноти
- визначає, до яких груп ароматів належить запит
- будує правильні уточнюючі питання, якщо запит нечіткий

### Системний prompt для LLM
Цей prompt навчає модель:
- коли відповідати “ok”
- коли ставити уточнення
- як формувати запитання
- які стилістичні обмеження використовувати

Функція для уточнення запиту у випадку пошуку по схожості до парфуму

### check_user_query
Цей блок оцінює запит користувача й вирішує:
- Чи користувач уже вказав конкретні ноти → повертає "ok"
- Чи це емоційний опис → формує уточнююче питання
- Чи запит занадто нечіткий → надсилає prompt у Claude
- Обробляє помилки → дає fallback питання

якщо користувач ввів назву парфуму шукаємо чи є вона у нашій колекції

якщо користувач хоче схожий аромат на заданий, витягуємо опис заданого парфуму

перетворює текст користувача на вектор

- Шукаємо у колекції Weaviate найбільш схожі вектори за полем desc_embedding до "вектору користувача"
- Обмежуємо кількість результатів топ‑k (за замовчуванням 5)
- Повертаємо список об’єктів з властивостями url і description

- Приймає знайдені результати з Weaviate і їхні описи
- Перетворює кожен опис у вектор за допомогою моделі
- Обчислює cosine similarity між вектором користувача і кожним описом
- Сортує результати за схожістю від найвищої до найнижчої.
- Повертає список об’єктів, від найрелевантнішого до найменш релевантного

### Генерація відповіді

- Перевіряє, чи є результати для запиту
- Формує контекст із топ-K знайдених ароматів (URL + опис)
- Створює чітку RAG-підказку для моделі
- Відправляє запит до OpenRouter / Claude
- Отримує і повертає відповідь користувачу
- Обробляє помилки, якщо запит не вдався

### Фінальна функція

## Оновлений варіант кейсу з пошуком по схожості до певного парфуму

In [ ]:
import json
import requests

EXTRACT_NAME_PROMPT = """
Ти — експерт з парфумерії.

Завдання:
1. Витягнути з тексту користувача НАЗВУ та БРЕНД парфуму, навіть якщо є орфографічні помилки.
2. Якщо назва написана приблизно — виправити її.
3. Якщо назви немає зовсім або вона нечітка — повернути JSON зі status="fail" і коротким уточненням.
4. Відповідь строго у JSON.

Успішно:
{
  "status": "ok",
  "brand": "Tom Ford",
  "name": "Lost Cherry"
}

Потрібне уточнення:
{
  "status": "fail",
  "error_sms": "Уточніть, будь ласка, назву аромату."
}

Важливо:
• НЕ вигадуй неіснуючих ароматів.
• Якщо можеш здогадатися, що мав на увазі користувач — виправ правильно.
• Відповідь ТІЛЬКИ JSON.
"""

def extract_perfume_name_AI_clean(user_text: str, OPENROUTER_KEY: str) -> dict:
    headers = {
        "Authorization": f"Bearer {OPENROUTER_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "anthropic/claude-3.5-sonnet",
        "temperature": 0.1,
        "max_tokens": 200,
        "messages": [
            {"role": "system", "content": EXTRACT_NAME_PROMPT},
            {"role": "user", "content": user_text}
        ]
    }

    try:
        r = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data,
            timeout=12
        )
        r.raise_for_status()
        raw = r.json()["choices"][0]["message"]["content"].strip()
        # можна залогувати, якщо щось піде не так
        # print("RAW NAME EXTRACTION:", raw)
        return json.loads(raw)

    except Exception as e:
        print("NAME EXTRACTION ERROR:", e)
        return {
            "status": "fail",
            "error_sms": "Не вдалося розпізнати назву. Напишіть, будь ласка, точніше."
        }

In [ ]:
NOTES_FROM_AI_PROMPT = """
Ти — парфумерний експерт.

Завдання:
1. Для аромату {brand} {name} поверни тільки:
   - верхні ноти
   - середні ноти
   - стать аромату (female / male / unisex)

Поверни STRICT JSON:
{
  "top": ["..."],
  "middle": ["..."],
  "gender": "female" | "male" | "unisex"
}

Ніяких пояснень.
"""

def get_notes_from_ai(brand, name, OPENROUTER_KEY):
    prompt = NOTES_FROM_AI_PROMPT.format(brand=brand, name=name)

    headers = {
        "Authorization": f"Bearer {OPENROUTER_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "anthropic/claude-3.5-sonnet",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.1
    }

    try:
        r = requests.post("https://openrouter.ai/api/v1/chat/completions",
                          headers=headers, json=data, timeout=15)
        raw = r.json()["choices"][0]["message"]["content"]
        return json.loads(raw)
    except:
        return None


In [ ]:
def extract_gender_from_description(desc: str):
    desc_low = desc.lower()

    if "for women" in desc_low or "women's" in desc_low or "female" in desc_low:
        return "female"

    if "for men" in desc_low or "men's" in desc_low or "male" in desc_low:
        return "male"

    if "unisex" in desc_low:
        return "unisex"

    return "unisex"


In [ ]:
def find_perfume_in_db_or_AI(brand, name, model, collection, OPENROUTER_KEY):
    query = f"{brand} {name}"
    vec = model.encode([query])[0].tolist()

    result = collection.query.near_vector(
        near_vector=vec,
        limit=1,
        return_properties=["url", "description"]
    )

    if result.objects:
        obj = result.objects[0]
        desc = obj.properties["description"]

        top = re.findall(r"Top notes are ([^.]+)", desc)
        top = top[0].split(", ") if top else []

        middle = re.findall(r"Middle notes are ([^.]+)", desc)
        middle = middle[0].split(", ") if middle else []

        gender = extract_gender_from_description(desc)

        return {
            "top": top,
            "middle": middle,
            "gender": gender,
            "source": "db"
        }

    else:
        return get_notes_from_ai(brand, name, OPENROUTER_KEY)


In [ ]:
def encode_notes(top, middle, model):
    combined = ", ".join(top + middle)
    return model.encode([combined])[0].tolist()

In [ ]:
def search_by_gender(vector, gender, collection, k=20):

    allowed = ["unisex"]
    if gender == "female":
        allowed += ["female"]
    elif gender == "male":
        allowed += ["male"]
    else:
        allowed = ["male", "female", "unisex"]

    result = collection.query.near_vector(
        near_vector=vector,
        target_vector="desc_embedding",
        limit=k,
        return_properties=["url", "description"]
    )
    # Фільтрація по статі з опису:
    filtered = []
    for o in result.objects:
        desc = o.properties.get("description", "")
        gender_extracted = extract_gender_from_description(desc)

        if gender_extracted in allowed:
            filtered.append(o)
    return filtered

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def rerank_with_threshold(vector, candidates, model, threshold=0.39):
    scored = []
    for obj in candidates:
        desc = obj.properties["description"]
        dvec = model.encode([desc])[0].reshape(1, -1)
        sim = cosine_similarity([vector], dvec)[0][0]
        if sim >= threshold:
            scored.append((sim, obj))

    scored.sort(key=lambda x: x[0], reverse=True)
    return [o for _, o in scored]

In [ ]:

def generate_similarity_response(base_perfume_name, base_notes, results, gender_preference, model_name="anthropic/claude-3.5-sonnet", top_k=5):

    if not results:
        return "😕 На жаль, не знайшов подібних ароматів."

    context_docs = []
    for obj in results[:top_k]:
        url = obj.properties.get("url", "")
        desc = obj.properties.get("description", "")
        context_docs.append(f"{url}\n{desc}")

    context = "\n\n".join(context_docs)

    gender_text = {
        "male": "для чоловіків",
        "female": "для жінок",
        "unisex": "унісекс"
    }.get(gender_preference, "унісекс")

    rag_prompt = f"""
Ти — парфумерний експерт.

Базовий аромат:
{base_perfume_name}
Основні ноти:
{base_notes}

Стать аромату, яку шукає користувач: {gender_text}

Ось список знайдених у базі ароматів:
{context}

Твоє завдання:

• Обери 3–5 ароматів з вищим показником схожості(тобто в першій частині)
• Для кожного напиши ОДИН абзац строго у форматі:

1. Назва аромату та бренд
URL
1–2 речення простими словами про те, чому він схожий (стиль, характер, настрій, профіль).

• НЕ використовуй bullet points.
• НЕ роби хімічних аналізів.
• НЕ пиши списків нот.
• НЕ згадуй верхні/середні/базові ноти.
• Не пояснюй процес.
• НЕ додавай ароматів, яких немає в контексті.
• Формат відповіді має бути ідентичний у ВСІХ кейсах, навіть коли користувач просить інший гендер.
• Не вигадуй нічого поза переданим текстом.

Пиши українською.
"""

    headers = {
        "Authorization": f"Bearer {OPENROUTER_KEY}",
        "Content-Type": "application/json",
    }

    data = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": "Ти корисний ароматний асистент."},
            {"role": "user", "content": rag_prompt},
        ],
        "temperature": 0.15,
        "max_tokens": 600
    }

    r = requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers=headers,
        json=data,
        timeout=15
    )

    return "Відповідь:\n" + r.json()["choices"][0]["message"]["content"]


In [ ]:
def handle_user_message_similarity(user_text, model, collection, OPENROUTER_KEY):

    name_data = extract_perfume_name_AI_clean(user_text, OPENROUTER_KEY)

    if name_data["status"] == "fail":
        return name_data["error_sms"]

    brand = name_data["brand"]
    name = name_data["name"]

    perfume_data = find_perfume_in_db_or_AI(
        brand, name, model, collection, OPENROUTER_KEY
    )

    if not perfume_data:
        return "Не вдалося знайти ноти аромату. Спробуйте іншу назву."

    top = perfume_data["top"]
    middle = perfume_data["middle"]
    gender = perfume_data["gender"]

    vector = encode_notes(top, middle, model)

    candidates = search_by_gender(vector, gender, collection)

    ranked = rerank_with_threshold(vector, candidates, model)
    
    notes_text = ", ".join(top + middle)

    return generate_similarity_response(base_perfume_name=name, base_notes=notes_text, results=ranked, gender_preference=gender)


In [ ]:
# import re
# print(
#     handle_user_message_similarity(
#         "Я хочу щось схоже на Tom Ford Lost Cherry",
#         model_faster,
#         collection,
#         "",
#     )
# )

In [ ]:
# print(handle_user_message_similarity("Я хочу щось схоже на Tom Ford Lost Cherry але для чоловіків", model_faster, collection, OPENROUTER_KEY))

In [ ]:
# print(handle_user_message_similarity("Я хочу щось схоже на Tom Ford Lost Cherry але з шоколадом", model_faster, collection, OPENROUTER_KEY))

In [ ]:
# print(handle_user_message_similarity("Я хочу щось схоже на лост чері", model_faster, collection, OPENROUTER_KEY))

In [ ]:
# print(handle_user_message_similarity("Я хочу щось схоже на ", model_faster, collection, OPENROUTER_KEY))

In [ ]:
# print(handle_user_message_similarity("Я хочу щось схоже на лст чеі", model_faster, collection, OPENROUTER_KEY))

In [ ]:
# print(handle_user_message_similarity("Я хочу щось схоже нa tom ford", model_faster, collection, OPENROUTER_KEY))

In [ ]:
import sqlite3


def init_db():
    conn = sqlite3.connect("user_data.db")
    c = conn.cursor()

    c.execute(
        """
        CREATE TABLE IF NOT EXISTS user_profile (
            user_id INTEGER PRIMARY KEY,
            gender TEXT,
            liked_perfumes TEXT,
            liked_notes TEXT
        )
        """
    )

    conn.commit()
    conn.close()


init_db()

In [ ]:
def add_user(user_id, gender):
    conn = sqlite3.connect("user_data.db")
    c = conn.cursor()

    c.execute(
        """
        INSERT OR REPLACE INTO user_profile (user_id, gender, liked_perfumes, liked_notes)
        VALUES (?, ?, '', '')
        """,
        (user_id, gender),
    )

    conn.commit()
    conn.close()


def clear_user_data(user_id):
    conn = sqlite3.connect("user_data.db")
    c = conn.cursor()

    c.execute(
        """
        UPDATE user_profile
        SET liked_perfumes = '', liked_notes = ''
        WHERE user_id = ?
        """,
        (user_id,),
    )

    conn.commit()
    conn.close()


def get_user_profile(user_id):
    conn = sqlite3.connect("user_data.db")
    c = conn.cursor()

    c.execute(
        "SELECT liked_perfumes, liked_notes FROM user_profile WHERE user_id = ?",
        (user_id,),
    )
    row = c.fetchone()
    conn.close()

    if not row:
        return None

    perfumes, notes = row

    return {
        "liked_perfumes": perfumes.split(", ") if perfumes else [],
        "liked_notes": notes.split(", ") if notes else [],
    }


def add_liked_perfume(user_id, perfume):
    profile = get_user_profile(user_id)
    if not profile:
        return

    perfumes = set(profile["liked_perfumes"])
    perfumes.add(perfume)

    perfumes_str = ", ".join(perfumes)

    conn = sqlite3.connect("user_data.db")
    c = conn.cursor()
    c.execute(
        "UPDATE user_profile SET liked_perfumes = ? WHERE user_id = ?",
        (perfumes_str, user_id),
    )
    conn.commit()
    conn.close()


def add_liked_note(user_id, note):
    profile = get_user_profile(user_id)
    if not profile:
        return

    notes = set(profile["liked_notes"])
    notes.add(note)

    notes_str = ", ".join(notes)

    conn = sqlite3.connect("user_data.db")
    c = conn.cursor()
    c.execute(
        "UPDATE user_profile SET liked_notes = ? WHERE user_id = ?",
        (notes_str, user_id),
    )
    conn.commit()
    conn.close()


import sqlite3
def add_notes_list_to_db(user_id, notes_list):
    for note in notes_list:
        add_liked_note(user_id, note)

### Telegram

In [ ]:
NOTES_FROM_AI_PROMPT = """
Ти — парфумерний експерт.

Завдання:
1. Для аромату {brand} {name} поверни тільки:
   - верхні ноти
   - середні ноти
   - стать аромату (female / male / unisex)

Поверни STRICT JSON:
{{
  "top": ["..."],
  "middle": ["..."],
  "gender": "female" | "male" | "unisex"
}}

Ніяких пояснень.
"""


def get_all_notes(names, brands, OPENROUTER_KEY):
    all_notes = []

    for name, brand in zip(names, brands):
        prompt = NOTES_FROM_AI_PROMPT.format(brand=brand, name=name)

        headers = {
            "Authorization": f"Bearer {OPENROUTER_KEY}",
            "Content-Type": "application/json; charset=utf-8",
        }

        data = {
            "model": "anthropic/claude-3.5-sonnet",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.1,
        }

        try:
            r = requests.post(
                "https://openrouter.ai/api/v1/chat/completions",
                headers=headers,
                json=data,
                timeout=15,
            )
            r.raise_for_status()
            raw = r.json()["choices"][0]["message"]["content"]
            notes = json.loads(raw)

            all_notes.extend(notes.get("top", []))
            all_notes.extend(notes.get("middle", []))

        except Exception as e:
            print(f"Помилка отримання ноток для {brand} {name}: {e}")
            continue

    return list(set(all_notes))


OPENROUTER_KEY = (
    ""
)

names = ["Fitnessence", "Frangipane e Cocco", "XVII Baroque Russian Coriander"]
brands = ["Matriarch", "I Profumi di Firenze", "Clive Christian"]

In [ ]:
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton
from aiogram.fsm.context import FSMContext
from aiogram import types
from aiogram.filters import Command
import asyncio
from aiogram import Bot, Dispatcher, Router, types
from aiogram.filters import Command
from aiogram.fsm.context import FSMContext
from aiogram.fsm.state import State, StatesGroup
from aiogram.fsm.storage.memory import MemoryStorage
from aiogram.types import InlineKeyboardButton, InlineKeyboardMarkup

TOKEN = TOKEN

class Form(StatesGroup):
    waiting_for_gender = State()
    waiting_for_description = State()
    waiting_for_name = State()

router = Router()

def gender_kb():
    return ReplyKeyboardMarkup(
        keyboard=[[KeyboardButton(text="Жіноча"), KeyboardButton(text="Чоловіча")]],
        resize_keyboard=True,
        one_time_keyboard=True,
    )

@router.message(Command("start"))
async def cmd_start(message: types.Message, state: FSMContext):
    user_id = message.from_user.id
    user = get_user_profile(user_id)
    print(user)
    if user:
        await message.answer(
            "Привіт! 👋 Радий бачити тебе знову!", reply_markup=main_menu_kb()
        )
        return
    await message.answer(
        "Привіт! 👋\nОбери, будь ласка, свою стать:",
        reply_markup=gender_kb(),
    )
    await state.set_state(Form.waiting_for_gender)


def main_menu_kb():
    return InlineKeyboardMarkup(
        inline_keyboard=[
            [
                InlineKeyboardButton(
                    text="🌷 Пошук за назвою парфуму",
                    callback_data="search_by_description",
                )
            ],
            [
                InlineKeyboardButton(
                    text="✨ Підбір аромату за описом",
                    callback_data="search_by_name",
                )
            ],
            [
                InlineKeyboardButton(
                    text="✨ Підбір аромату за власними вподобаннями",
                    callback_data="search_by_user",
                )
            ],
            [InlineKeyboardButton(text="Про бота", callback_data="about")],
        ]
    )

@router.message(Form.waiting_for_gender)
async def process_gender(message: types.Message, state: FSMContext):
    gender = message.text.strip()

    if gender not in ["Жіноча", "Чоловіча"]:
        await message.answer(
            "Будь ласка, обери один з варіантів:", reply_markup=gender_kb()
        )
        return
    user_id = message.from_user.id
    add_user(user_id, gender)

    await state.update_data(gender=gender)
    await message.answer(f"Дякую! Обрана стать: {gender}.", reply_markup=main_menu_kb())
    await state.clear()


@router.message(Command("help"))
async def cmd_help(message: types.Message):
    await message.answer(
        "Команди:\n"
        "/start — почати\n"
        "/help — допомога\n\n"
        "Надішли будь-який текст — я відповім ехо."
    )


@router.callback_query(lambda c: c.data == "about")
async def cb_about(callback: types.CallbackQuery):
    await callback.message.answer(
        "ℹ️ Цей бот створено на aiogram 3.3.0.\nВін показує приклад кнопок і FSM."
    )
    await callback.answer()

# Підбір за назвою

In [ ]:
import re

def feedback_kb():
    return InlineKeyboardMarkup(
        inline_keyboard=[
            [
                InlineKeyboardButton(text="🌟 Сподобалось", callback_data="liked"),
                InlineKeyboardButton(
                    text="❌ Не сподобалось", callback_data="disliked"
                ),
            ],
            [InlineKeyboardButton(text="🏠 Головне меню", callback_data="main_menu")],
        ]
    )

async def send_recommendations(
    obj,
    state: FSMContext,
    text: str,
):
    perfume_names = extract_perfume_names(text)

    await state.update_data(last_recommendations=perfume_names)

    if isinstance(obj, types.Message):
        await obj.answer(text)
        await obj.answer("Оберіть, будь ласка:", reply_markup=feedback_kb())
    else:
        await obj.message.answer(text)
        await obj.message.answer("Оберіть, будь ласка:", reply_markup=feedback_kb())

    await state.set_state(Form.waiting_for_description)


OPENROUTER_KEY = (
    ""
)
def get_all_notes(names, brands, OPENROUTER_KEY):
    all_notes = []

    for name, brand in zip(names, brands):
        prompt = NOTES_FROM_AI_PROMPT.format(brand=brand, name=name)

        headers = {
            "Authorization": f"Bearer {OPENROUTER_KEY}",
            "Content-Type": "application/json; charset=utf-8",
        }

        data = {
            "model": "anthropic/claude-3.5-sonnet",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.1,
        }

        try:
            r = requests.post(
                "https://openrouter.ai/api/v1/chat/completions",
                headers=headers,
                json=data,
                timeout=15,
            )
            r.raise_for_status()
            raw = r.json()["choices"][0]["message"]["content"]
            notes = json.loads(raw)

            all_notes.extend(notes.get("top", []))
            all_notes.extend(notes.get("middle", []))

        except Exception as e:
            print(f"Помилка отримання ноток для {brand} {name}: {e}")
            continue

    return list(set(all_notes))

def extract_perfume_names(text: str):
    pattern = pattern = r"(?:[:.\n]\s*)(.+?)\s+від\b"
    names = re.findall(pattern, text)
    return names

def extract_perfume_brands(text: str):
    pattern = r"від\s+(.+?)\s+https"
    brands = re.findall(pattern, text)
    return brands

@router.message(Form.waiting_for_description)
async def process_description(message: types.Message, state: FSMContext):
    full_prompt = message.text.strip()
    print(full_prompt)
    recs = handle_user_message_similarity(
        full_prompt, model_faster, collection, OPENROUTER_KEY
    )
    perfume_names = extract_perfume_names(recs)
    perfume_brands = extract_perfume_brands(recs)
    all_notes_list = get_all_notes(perfume_names, perfume_brands, OPENROUTER_KEY)
    add_notes_list_to_db(message.from_user.id, all_notes_list)

    print(all_notes_list)
    await send_recommendations(message, state, recs)

@router.callback_query(lambda c: c.data == "liked")
async def cb_liked(callback: types.CallbackQuery, state: FSMContext):
    data = await state.get_data()
    perfume_names = data.get("last_recommendations", [])
    user_id = callback.from_user.id

    if not perfume_names:
        await callback.message.answer("❗ Немає ароматів для збереження.")
        await callback.answer()
        return

    for perfume in perfume_names:
        add_liked_perfume(user_id, perfume)

    perfume_brands = [""] * len(perfume_names)

    notes = get_all_notes(
        names=perfume_names, brands=perfume_brands, OPENROUTER_KEY=OPENROUTER_KEY
    )

    add_notes_list_to_db(user_id, notes)

    await callback.message.answer("✅ Додано улюблені аромати та їхні ноти 🌸")
    await state.clear()
    await callback.answer()

@router.callback_query(lambda c: c.data == "disliked")
async def cb_disliked(callback: types.CallbackQuery, state: FSMContext):
    await callback.message.answer("❌ Добре, нічого не зберігаємо.")
    await state.clear()
    await callback.answer()

@router.callback_query(lambda c: c.data == "main_menu")
async def cb_main_menu(callback: types.CallbackQuery, state: FSMContext):
    await callback.message.answer("🏠 Головне меню", reply_markup=main_menu_kb())
    await state.clear()
    await callback.answer()

@router.callback_query(lambda c: c.data == "search_by_description")
async def cb_search_description(callback: types.CallbackQuery, state: FSMContext):
    await callback.message.answer("Введи назву аромату, схожий до якого бажаєш знайти:")
    await state.set_state(Form.waiting_for_description)
    await callback.answer()

# Підбір за вподобаннями користувача

In [ ]:
@router.callback_query(lambda c: c.data == "search_by_user")
async def cb_search_by_user(callback: types.CallbackQuery, state: FSMContext):
    user_id = callback.from_user.id

    user_data = get_user_profile(user_id)
    liked_notes = user_data.get("liked_notes", []) if user_data else []

    if not liked_notes:
        await callback.message.answer(
            "❗ У вас ще немає улюблених ноток. Спершу оберіть аромати, щоб ми могли підбирати схожі."
        )
        await callback.message.answer(
            "Оберіть наступну дію:", reply_markup=main_menu_kb()
        )
        await callback.answer()
        return

    notes_prompt = f"Підбери парфуми які містять такі нотки: {', '.join(liked_notes)}. У своїй відповіді не вказуй ноти, які відсутні в парфумах, які ти рекомендуєш"

    recommended_perfumes = handle_user_message_kate(
        message=notes_prompt,
        model=model_faster,
        collection=collection,
    )

    perfume_names = extract_perfume_names(recommended_perfumes)
    perfume_brands = extract_perfume_brands(recommended_perfumes)
    all_notes_list = get_all_notes(perfume_names, perfume_brands, OPENROUTER_KEY)
    add_notes_list_to_db(callback.from_user.id, all_notes_list)

    print(all_notes_list)
    await send_recommendations(callback, state, recommended_perfumes)

# Підбір за нотками

In [ ]:
@router.message(Form.waiting_for_name)
async def process_name(message: types.Message, state: FSMContext):
    name = message.text.strip()
    if not name:
        await message.answer("❗ Будь ласка, введи назву парфуму ще раз.")
        return

    answer = handle_user_message_kate(
        message=name,
        model=model_faster,
        collection=collection,
    )

    perfume_names = extract_perfume_names(answer)
    perfume_brands = extract_perfume_brands(answer)
    all_notes_list = get_all_notes(perfume_names, perfume_brands, OPENROUTER_KEY)
    add_notes_list_to_db(message.from_user.id, all_notes_list)

    print(all_notes_list)
    await send_recommendations(message, state, answer)


@router.callback_query(lambda c: c.data == "search_by_name")
async def cb_search_name(callback: types.CallbackQuery, state: FSMContext):
    await callback.message.answer("Введіть опис бажаного аромату:")
    await state.set_state(Form.waiting_for_name)
    await callback.answer()

In [ ]:
@router.message()
async def echo(message: types.Message):
    if message.text and not message.text.startswith("/"):
        await message.answer(f"Ехо: {message.text}")
    else:
        await message.answer("Невідома команда. Спробуй /help")

@router.callback_query(lambda c: c.data == "search_by_name_with_notes")
async def cb_search_name_with_notes(callback: types.CallbackQuery):
    await callback.message.answer(
        "Введи назву парфуму, а потім додаткові нотки, які хочеш додати чи виключити:"
    )
    await callback.answer()

async def main():
    bot = Bot(token="")
    dp = Dispatcher(storage=MemoryStorage())
    dp.include_router(router)
    print("✅ Бот запущений і працює!")
    await dp.start_polling(bot)

if __name__ == "__main__":
    await main()